In [149]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import ADASYN
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

In [150]:
def loading_datasets():
    start_time = time.time()
    print("------------------------")
    print("Загрузка данных началась")
    
    #Загружаем датасеты
    train = pd.read_csv(r'C:\datasets\Сбербанк\train_2.csv', low_memory=False)
    test = pd.read_csv(r'C:\datasets\Сбербанк\test_2.csv', low_memory=False)
    
    train.set_index('ticket_id', inplace=True)
    test.set_index('ticket_id', inplace=True)
    
    print("Загрузка данных закончена за - %s seconds - OK" % (time.time() - start_time))
    
    return train, test

In [151]:
def preprocessing_datasets(train, test):
    start_time = time.time()
    print("-----------------------------")
    print("Предобработка данных началась")
    
    #Удалим признаки, которые нам не понадобятся для обучения модели
    train = train.drop('Unnamed: 0', axis=1) #Артефакт
    test = test.drop('Unnamed: 0', axis=1) # Артефакт
    print("    Размер train до предобработки: {}".format(train.shape))
    print("    Размер test до предобработки: {}".format(test.shape))
    
    #Оставим в train признаки, присутствующие в test
    columns_outside_test = set(train) - set(test) - {'compliance'}
    total_columns = set(train) - columns_outside_test
    train = train[total_columns]
    
    #Удалим неинформативные для модели признаки (объясним почему):
    #grafitti_status, non_us_str_code, violation_zip_code удалям, так как по данным признакам нет значений
    #Tак как признак с grafitti_status у нас отсутствует в данных, то и нет смысла в признаке clean_up_cost
    #violation_description - это описание violation_code
    #country - не информативен, 99,99% признаков имеют одно значение
    #admin_fee, late_fee, state_fee и fine_amount в сумме являются judgment_amount за минусом скидки
    #Адреса, имя инспектора, название компании выписавшей штраф и тд - слишком частные даные, не несут
    #общей обобщающей способности
    delete_columns = ['grafitti_status', 'non_us_str_code', 'violation_zip_code', 'clean_up_cost', 
                      'violation_description', 'country', 'admin_fee', 'late_fee', 'state_fee', 
                      'fine_amount', 'violator_name', 'city', 'zip_code', 'mailing_address_str_name', 
                      'state', 'inspector_name', 'violation_street_number', 'agency_name', 
                      'mailing_address_str_number', 'violation_street_name']
    train = train.drop(delete_columns, axis=1)
    test = test.drop(delete_columns, axis=1)
    
    #Удаляем события в пропусках целевой переменной
    train = train.dropna(subset=['compliance'])
    
    #-----------------------------------------------------------------
    #Удалим из violation_code всё что за спецсимволами, так как это уточнение, а нам необходимы обобщающие признаки
    def delete_special_case(data):
        data['violation_code'] = data['violation_code'].apply(lambda x: x.split('(')[0])
        data['violation_code'] = data['violation_code'].apply(lambda x: x.split('/')[0])
        data['violation_code'] = data['violation_code'].apply(lambda x: x.split(' ')[0])
        data['violation_code'][data['violation_code'].apply(lambda x: x.find('-')<=0)] = ''
    delete_special_case(train)
    delete_special_case(test)
    
    #Cтатьи, по которым меньше 100 правонарущений объеденим в одну категорию - Другие
    counts = train['violation_code'].value_counts()
    train['violation_code'][train['violation_code'].isin(counts[counts < 100].index)] = 'Other'
    test['violation_code'][test['violation_code'].isin(counts[counts < 100].index)] = 'Other'
    
    #Категоризируем violation_code по частоте его выписывания, для этого используем словарь
    violation_code_mean = pd.DataFrame(data=[train.groupby('violation_code')['violation_code'].count().index, 
                       train.groupby('violation_code')['violation_code'].count()]).T
    violation_code_mean[1] = violation_code_mean[1] / len(train)
    violation_code_dict = {x[0] : x[1] for x in violation_code_mean.itertuples(index=False)}
    
    def violation_code_categorizer(row):
        try:
            revenue = violation_code_dict[row['violation_code']]
        except:
            revenue = violation_code_dict['Other']
        return revenue
    
    test['violation_code'] = test.apply(violation_code_categorizer, axis=1)
    train['violation_code'] = train.apply(violation_code_categorizer, axis=1)
    
    #-----------------------------------------------------------------
    #Предобработаем данные с датами
    train.ticket_issued_date = pd.to_datetime(train.ticket_issued_date)
    test.ticket_issued_date = pd.to_datetime(test.ticket_issued_date)
    train.hearing_date = pd.to_datetime(train.hearing_date)
    test.hearing_date = pd.to_datetime(test.hearing_date)
    #Введем новый признак, количество дней от штрафа до суда
    train['timedelta'] = (train.hearing_date - train.ticket_issued_date).dt.days
    test['timedelta'] = (test.hearing_date - test.ticket_issued_date).dt.days
    #Теперь выделим из признака даты: месяц, число, день недели. 
    #К дню недели прибавим 1, чтобы не было 0. В данных, где судебное решение не назначено, 
    #установим значение -1, чтобы модель могла более корректно отработать, 0 означает 0, а -1 - это значение
    train['issued_day'] = train.ticket_issued_date.dt.day
    train['issued_month'] = train.ticket_issued_date.dt.month
    train['issued_weekday'] = train.ticket_issued_date.dt.weekday+1
    train['hearing_day'] = train.hearing_date.dt.day
    train['hearing_month'] = train.hearing_date.dt.month
    train['hearing_weekday'] = train.hearing_date.dt.weekday+1

    test['issued_day'] = test.ticket_issued_date.dt.day
    test['issued_month'] = test.ticket_issued_date.dt.month
    test['issued_weekday'] = test.ticket_issued_date.dt.weekday+1
    test['hearing_day'] = test.hearing_date.dt.day
    test['hearing_month'] = test.hearing_date.dt.month
    test['hearing_weekday'] = test.hearing_date.dt.weekday+1

    train = train.drop(['ticket_issued_date', 'hearing_date'], axis=1)
    train = train.fillna(-1)

    test = test.drop(['ticket_issued_date', 'hearing_date'], axis=1)
    test = test.fillna(-1)
    #--------------------------------------------------------------------
    
    #В признаке disposition мало уникальных значений, разложим через One Hot
    encoder = OneHotEncoder(sparse=False)

    disposition_one_hot_train = train['disposition'].values.reshape(-1, 1)
    disposition_one_hot_test = test['disposition'].values.reshape(-1, 1)

    encoder.fit(disposition_one_hot_train)
    encoder.fit(disposition_one_hot_test)

    train_disposition = pd.DataFrame(encoder.transform(disposition_one_hot_train), 
                                 columns=encoder.categories_, 
                                 index=train.index)
    train = train.drop('disposition', axis=1)

    test_disposition = pd.DataFrame(encoder.transform(disposition_one_hot_test), 
                                columns=encoder.categories_, 
                                index=test.index)
    test = test.drop('disposition', axis=1)
    train = train.merge(train_disposition, left_index=True, right_index=True)
    test = test.merge(test_disposition, left_index=True, right_index=True)
    #-------------------------------------------------------------------------
    #Предположим, что вероятность уплаты штрафа каким то образом может зависеть от района. 
    #Кластеризируем координаты адресов правонарушений
    #Я хотел использовать DBSCAN, но у меня не хватает оперативной памяти на стационарном компьютере, 
    #чтобы провести данную кластеризацию, поэтому я проведу кластеризацию KMean, с разбивкой на 100 районов.
    start_time_cluster = time.time()
    print("    -----------------------------")
    print("    Начало кластеризации адресов...")
    latlon = pd.read_csv(r'C:\datasets\Сбербанк\latlons.csv', low_memory=False)
    addresses = pd.read_csv(r'C:\datasets\Сбербанк\addresses.csv', low_memory=False)
    
    latlon = latlon.dropna()
    lat_sc = latlon[['lat', 'lon']]
    db = KMeans(n_clusters=100)
    clusters = db.fit_predict(lat_sc)
    latlon = latlon.merge(pd.DataFrame(clusters, index=latlon.index), left_index=True, right_index=True)
    print("    Предобработка кластеризации завершена - %s seconds - OK" % (time.time() - start_time_cluster))
    print("    -----------------------------")
    
    #Добавим результат кластеризации в обучающий и тестовый датасеты
    total_ad = latlon.merge(addresses)
    total_ad.set_index('ticket_id', inplace=True)
    train = train.merge(total_ad, left_index=True, right_index=True)
    test = test.merge(total_ad, left_index=True, right_index=True)
    train = train.drop(['address', 'lat', 'lon'], axis=1)
    test = test.drop(['address', 'lat', 'lon'], axis=1)
    print("    Размер train после предобработки: {}".format(train.shape))
    print("    Размер test после предобработки: {}".format(test.shape))
    
    #Перемешаем данные
    train = shuffle(train)
    
    #Удалим дубликаты
    train.drop_duplicates(inplace=True)
    
    print("Предобработка данных за - %s seconds - OK" % (time.time() - start_time))
    
    return train, test

In [152]:
def learning_model(train, test):
    '''
    Для предсказаний будем использовать модель классификации из библиотеки CatBoost
    В черновике были проверены стандартные инструменты из sklearn(LogReg, SVC, RandomForest, GradientBoosting)
    они показали себя хуже
    
    Так же подпор модели был сделан с помощью Pipeline и GridSearchCV, в финальном варианте не буду их использовать,
    чтобы сэкономить процессорное время. Пользоваться данными инструментами умею.
    '''
    start_time = time.time()
    print("-----------------------------")
    print("Обучение модели...")
    
    #Загрузим модель
    model = CatBoostClassifier(silent=True)
    
    #Разобъем train на признаки и целевую переменную
    X = train.drop('compliance', axis=1)
    y = train['compliance']
    
    #В наших данных имеется дисбаланс классов, 
    #устраним его адаптивным дополнением данных в обучающем наборе при помощи библиотеки imblearn
    start_time_upsampling = time.time()
    print("    Баланс классов до upsampling")
    print("    |Класс 0 :{:.2%} | Класс 1 :{:.2%}|".format(train.compliance.value_counts(normalize=True)[0], 
                                           train.compliance.value_counts(normalize=True)[1]))
    imbalance = ADASYN(random_state=42)
    X_res, y_res = imbalance.fit_resample(X, y)
    X_res, y_res = shuffle(X_res, y_res)
    print("    upsampling завершен - %s seconds - OK" % (time.time() - start_time_upsampling))   
    
    #Проверим точность модели на кросс-валидации
    start_time_cv = time.time()
    print("    -------------------------------")
    print("    Кросс-валидация началась...")
    print("    ROC AUC на кросс-валидации: {:.2f}".format(cross_val_score(model, 
                                                                      X_res, y_res, 
                                                                      cv=5, 
                                                                      scoring='roc_auc')
                                                      .mean()))
    print("    Кросс-валидация завершена за - %s seconds - OK\n" % (time.time() - start_time_cv))
    
    print("    -------------------------------")
    start_time_fit = time.time()
    print("    Обучение модели на полных данных...")
    model.fit(X_res, y_res)
    print("    Обучение завершено за - %s seconds - OK\n" % (time.time() - start_time_fit))
    
    print("    -------------------------------")
    start_time_predict = time.time()
    print("    Предсказание на тестовых данных...")
    test_predict = model.predict_proba(test)[:, 1]
    print("    Предсказание завершено за - %s seconds - OK\n" % (time.time() - start_time_predict))
    
    test_predict = pd.DataFrame(data=[test.index, test_predict]).T
    test_predict.columns = ['ticket_id', 'probablity']
    test_predict.ticket_id = test_predict.ticket_id.astype('int64')
    
    print("Обучение завершено за - %s seconds - OK" % (time.time() - start_time))
    
    return test_predict

In [153]:
def run_predict():
    train, test = loading_datasets()
    train, test = preprocessing_datasets(train, test)
    test_predict = learning_model(train, test)
    test_predict.to_csv(r'C:\datasets\Сбербанк\test_predict.csv')

In [154]:
run_predict()

------------------------
Загрузка данных началась
Загрузка данных закончена за - 2.3001315593719482 seconds - OK
-----------------------------
Предобработка данных началась
    Размер train до предобработки: (225000, 33)
    Размер test до предобработки: (25305, 26)
    -----------------------------
    Начало кластеризации адресов...
    Предобработка кластеризации завершена - 122.41600179672241 seconds - OK
    -----------------------------
    Размер train после предобработки: (144526, 21)
    Размер test после предобработки: (25304, 20)
Предобработка данных за - 128.6503586769104 seconds - OK
-----------------------------
Обучение модели...
    Баланс классов до upsampling
    |Класс 0 :90.44% | Класс 1 :9.56%|
    upsampling завершен - 10.829619407653809 seconds - OK
    -------------------------------
    Кросс-валидация началась...
    ROC AUC на кросс-валидации: 0.98
    Кросс-валидация завершена за - 273.9536693096161 seconds - OK

    -------------------------------
    Обуче